# Исходные данные – корпус новостей с сайта Lenta.ru (https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/). Нужно обучить классификатор новостей по рубрикам (поле topic), для чего: 
### 1.    Предобработать тексты и получить признаковое пространство
### 2.    Выбрать модель или несколько   моделей для обучения
### 3.    Разделить датасет на обучающую, тестовую и контрольную выборки
### 4.    Выбрать метрику для оценки результатов

## 1. Т.к. наша задача сводиться к многоклассовой классификации текста по темам, то нужно перевести текст в векторной пространство и каждому тексту сопоставить номер(уникальный индентификатор) темы.
## 2. Использовал только одну модель, т.к. при подборе параметов ч\з "решето" уходит очень много вычислительного времени
## 3.Использовал деление на обучающую и тестовую выборки в связи с тем, что использовал "решето" для подбора лучших параметров и кросс-валидации.
## 4. В качестве метрики выбрал точность классификации(у нас многоклассовая классификация)

In [34]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline
import nltk
import re
from nltk.stem.snowball import SnowballStemmer#стеминг
from sklearn.feature_extraction.text import TfidfVectorizer#tfidf векторизатор
from sklearn.ensemble import RandomForestClassifier#наша модель
from sklearn.metrics import accuracy_score #для оценки модели
from sklearn.model_selection import GridSearchCV # для определения лучшего параметра
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# 1. Загружаем данные и обрабатываем.
### Т.к. машина слабая, то я использую только первые 25к строк

In [2]:
number_rows = 25000
df = pd.read_csv('lenta-ru-news.csv', nrows = number_rows)

In [3]:
df.head()

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество


In [4]:
#Для упрощения работы, создаю переменные с именами столбцов к которым я буду часто обращаться
text_col = 'text'
topic_col = 'topic'

In [5]:
#Т.к. наша цель классификация тем(признак topic) 
#на основе текста(text), то остальные признаки не нужны
df.drop(['url', 'title', 'tags'], axis = 1, inplace = True)

In [6]:
#смотрю пропуски
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
text     25000 non-null object
topic    24992 non-null object
dtypes: object(2)
memory usage: 390.7+ KB


In [7]:
# есть текст, но нету темы. 
#Т.к. в общей совокупности данных доля пропусков очень мала, то я их удаляю
df[df[topic_col].isnull()]

,text,topic
3530,в учебном центре врачебной практики praxi medi...,NaN
8736,организаторы выставки «игромир» и фестиваля co...,NaN
9956,российская студия разработчиков enplex games р...,NaN
10308,современный рынок товаров и услуг настолько ра...,NaN
16333,оргкомитет восточного экономического форума на...,NaN
17562,фонд росконгресс и федерация торгово-промышлен...,NaN
22497,сша предъявили кндр требования по 47 пунктам в...,NaN
24522,в красноярском крае задержали подозреваемого в...,NaN


In [8]:
df.dropna(inplace = True)

In [9]:
df.info() #пропусков нету

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24992 entries, 0 to 24999
Data columns (total 2 columns):
text     24992 non-null object
topic    24992 non-null object
dtypes: object(2)
memory usage: 585.8+ KB


In [11]:
'''
Подготавливаю данные для векторизации. 
Перевожу в нижний регистр и удаляю все знаки препинания
'''
df[text_col] = df[text_col].str.lower()
df[topic_col] = df[topic_col].str.lower()
df[text_col] = df[text_col].str.replace(',', ' ')
df[text_col] = df[text_col].str.replace('.', ' ')
df[text_col] = df[text_col].str.replace('-', ' ')
df[text_col] = df[text_col].str.replace(';', ' ')
df[text_col] = df[text_col].str.replace(':', ' ')
df[text_col] = df[text_col].str.replace('(', ' ')
df[text_col] = df[text_col].str.replace(')', ' ')
df[text_col] = df[text_col].str.replace(r'[\W]+', ' ')

In [12]:
df.head()

,text,topic
0,вице премьер по социальным вопросам татьяна го...,россия
1,австрийские правоохранительные органы не предс...,спорт
2,сотрудники социальной сети instagram проанализ...,путешествия
3,с начала расследования российского вмешательст...,мир
4,хакерская группировка anonymous опубликовала н...,мир


In [14]:
#функция токенизации и стемминга
def token_and_stem(text):
    tokens  = [word for word in nltk.word_tokenize(text)] # выделяем список слов из текста
    #print(tokens, '\n')
    filt_tokens = []# очищенный список слов без цифр и которые состоят минимум из 4 символов
    for token in tokens:
        if re.search('[а-яА-Яa-zA-Z]', token): #убираем цифры
            if len(token) > 3: #смотрим, что бы слова состояли минимум из 4 символов
                filt_tokens.append(token)
    #print(filt_tokens, '\n')
    stems = [] #стемминг
    for token in filt_tokens:
        if re.search('[а-яА-Я]', token): #для русских слов стемминг
            stems.append(stemmer_rus.stem(token))
        elif re.search('[a-zA-Z]', token):#для английских слов стемминг
            stems.append(stemmer_eng.stem(token))
    #print(stems)
    return stems

In [15]:
#создаю стемминги для рус и англ языка отдельно
stemmer_rus = SnowballStemmer('russian')
stemmer_eng = SnowballStemmer('english')

In [16]:
#создаем стоп слова из англ и русского алфавита
file_name_stopwords = 'stopwords-ru.txt' #нашел в интернете русские стоп-слова 
f = open(file_name_stopwords, encoding='utf8').read()
stopwords_rus = f.split('\n') #русские стоп-слова
stopwords_eng = nltk.corpus.stopwords.words('english') #англ. стоп-слова
stopwords = [] #общий список стоп-слов который объединяет русс-ие и англ-ие 
stopwords.extend(stopwords_rus)
stopwords.extend(stopwords_eng)

# Векторизация
### Использую для векторизации tf-idf метод который прописан в библиотеке. 
### Выбор данного метода мотивирован скоростью вычисления при минимальных требованиях оборудования. 

In [17]:
#создаю векторизатор
tfidf_vectorizer = TfidfVectorizer(max_df= 0.9, max_features= 1000, min_df= 0.05, 
                                  stop_words= stopwords, use_idf= True, tokenizer=token_and_stem)
#с помощью векторизатора перевожу в векторное пространство текст
tfidf_matrix = tfidf_vectorizer.fit_transform(df[text_col]) 
print(tfidf_matrix.shape)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'becaus', 'befor', 'could', 'doe', 'dure', 'might', 'must', 'need', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'would', 'yourselv', 'алл', 'бел', 'близк', 'бол', 'больш', 'буд', 'будеш', 'будт', 'быв', 'быва', 'быт', 'важн', 'вдал', 'ве', 'вед', 'везд', 'вернут', 'взят', 'видет', 'вмест', 'вод', 'войн', 'вообщ', 'восем', 'восемнадца', 'восемнадцат', 'восьм', 'впроч', 'врем', 'времен', 'всег', 'всегд', 'всюд', 'втор', 'выйт', 'главн', 'говор', 'голов', 'дава', 'давн', 'даж', 'далек', 'дальш', 'дар', 'дат', 'двадца', 'двадцат', 'двенадца', 'двенадцат', 'двер', 'девя', 'девят', 'девятнадца', 'девятнадцат', 'действительн', 'дела', 'ден', 'деньг', 'деся', 'десят', 'довольн', 'долг', 'долж', 'должн', 'дорог', 'дума', 'душ', 'есл', 'ест', 'ждат', 'жен', 'женщин', 'жизн', 'жит', 'зан', 

(24992, 294)


In [18]:
df[topic_col].nunique() # смотрю кол-о уникальных тем

17

In [39]:
# записываю все уникальные темы в массив topic_value
topic_value = df[topic_col].unique()
topic_value

array(['россия', 'спорт', 'путешествия', 'мир', 'бывший ссср',
       'интернет и сми', 'силовые структуры', 'экономика', 'культура',
       'дом', 'наука и техника', 'из жизни', 'ценности', 'бизнес',
       '69-я параллель', 'культпросвет ', 'крым'], dtype=object)

In [21]:
# создаю еще один признак в котором будет написан номер темы
# таким образом каждой теме соот-т свой уникальный индентификатор
for i, val in enumerate(topic_value):
    mask = df[topic_col] == val
    df.loc[mask, 'topic_class'] = int(i)

In [22]:
df.head()

,text,topic,topic_class
0,вице премьер по социальным вопросам татьяна го...,россия,0.0
1,австрийские правоохранительные органы не предс...,спорт,1.0
2,сотрудники социальной сети instagram проанализ...,путешествия,2.0
3,с начала расследования российского вмешательст...,мир,3.0
4,хакерская группировка anonymous опубликовала н...,мир,3.0


In [23]:
# стратифицированное(по признаку "topic_class") деление данных на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['topic_class'], 
                                                    test_size = 0.3,
                                                    stratify = df['topic_class'],
                                                    random_state = 42)

In [24]:
X_train.shape

(17494, 294)

# Построение модели
### Выбор модели "случайный лес" обоснован тем, что у нас многоклассовая классификация

In [25]:
#использую "решето" для подбора наилучего кол-а деревьев и глубины
r_forest = RandomForestClassifier(random_state = 42)
num_forest = [10, 100, 1000] # возможные варианты кол-а деревьев
num_depth = [5, 15, 25]# возможные варианты глубины деревьев
# "решетка" с кросс-вал проверкой. Использую 5 фолдов для кросс проверки и использую все ядра процессора
grid = GridSearchCV(r_forest, param_grid={'n_estimators': num_forest, 'max_depth': num_depth}, 
                    cv = 5, n_jobs= -1) 
#обучаю "решетку"
grid.fit(X_train, y_train)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [10, 100, 1000], 'max_depth': [5, 15, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
cv_err = 1 - grid.best_score_ #наша кросс-валидаточная ошибка на обучающей выборке
best_num_forest = grid.best_estimator_.n_estimators # птимальное кол-о деревьев
best_num_depth = grid.best_estimator_.max_depth # Оптимальная глубина деревьев
print('Ошибка перекрестной проверки:', cv_err)
print('Оптимальное кол-о деревьев:', best_num_forest)
print('Оптимальная глубина деревьев:', best_num_depth)

Ошибка перекрестной проверки: 0.35229221447353376
Оптимальное кол-о деревьев: 1000
Оптимальная глубина деревьев: 25


In [27]:
#строю модель с оптимальными параметрами
best_r_forest = RandomForestClassifier(n_estimators = best_num_forest, 
                                       max_depth = best_num_depth,
                                       random_state = 42)

In [28]:
best_r_forest.fit(X_train, y_train) # обучаю

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [35]:
y_pred_train = best_r_forest.predict(X_train) # классификация на обучающей выборке
y_pred_test = best_r_forest.predict(X_test) # классификация на тестовой выборке

In [38]:
print('Точность на обучающей выборке: {:.3f}'.format(accuracy_score(y_train, y_pred_train)))
print('Точность на тестовой выборке: {:.3f}'.format(accuracy_score(y_test, y_pred_test)))

Точность на обучающей выборке: 0.984
Точность на тестовой выборке: 0.660


## Низкая точность возможна обусловлена тем, что нужно подобрать лучшие параметры для векторизации текста(токенизация, стемминг, параметры векторизатора tfidf). Можно зациклить весь этот процесс(веторизация текста и подбор моделей, параметров модели) и в каждом цикле проверять точность на валидационной выборке(При этом разделить изначально данные на обучающую, валидационную и тестовую). Затем с лучшей матрицей текста, моделью и его лучшими параметрами посторить модель и провести классификация на тестовой выборке.